# Advent of Code 2024 Day 20 

### Setup

In [68]:
from aocd import get_data, submit

day = 20
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
import numpy as np

def parse_data(raw_data):
    lines = raw_data.splitlines()
    
    data = [list(line) for line in lines]
    
    return np.array(data)

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data[0]

### Part One!

In [72]:
use_sample_data = False
part = 'a'

In [73]:
data = sample_data if use_sample_data else test_data

In [74]:
from typing import Literal, Union

Track = Literal['.']
Wall = Literal['#']
Start = Literal['S']
End = Literal['E']

Tile = Union[Track, Wall, Start, End]

In [75]:
from functools import total_ordering

@total_ordering
class KeyDict(object):
    def __init__(self, key, dct):
        self.key = key
        self.dct = dct

    def __lt__(self, other):
        return self.key < other.key

    def __eq__(self, other):
        return self.key == other.key

    def __repr__(self):
        return '{0.__class__.__name__}(key={0.key}, dct={0.dct})'.format(self)
    
    def __iter__(self):
        return iter((self.key, self.dct))
    
    def __getitem__(self, key):
        return self.dct[key]
    
    def __setitem__(self, key, value):
        self.dct[key] = value

In [76]:
def get_candidate_positions(arr:np.ndarray, pos:np.ndarray, remove_walls:bool=True):
    directions = [
        [0, 1],
        [0, -1],
        [1, 0],
        [-1, 0]
    ]
    
    for dir in directions:
        candidate_pos = pos + dir
        if remove_walls and arr[tuple(candidate_pos)] == '#':
            continue
        elif candidate_pos[0] < 0 or candidate_pos[1] < 0:
            continue
        elif candidate_pos[0] >= arr.shape[0] or candidate_pos[1] >= arr.shape[1]:
            continue
        
        yield np.array(candidate_pos)

In [77]:
def euclidean_distance(a:np.ndarray, b:np.ndarray):
    return np.sqrt(np.sum((a - b)**2))

def manhattan_distance(a:np.ndarray, b:np.ndarray):
    return np.sum(np.abs(a - b))

def dummy_heuristic(a:np.ndarray, b:np.ndarray):
    return 0

In [78]:
def cost (arr:np.ndarray, a:np.ndarray, b:np.ndarray):
    return 1

In [79]:
import heapq

def astar(arr:np.ndarray, start:np.ndarray, end:np.ndarray, heuristic_fn = manhattan_distance, cost_fn = cost):
    queue = [KeyDict(0, { 'path': [start] })]
    visited = set()
    
    while queue:
        score, current = heapq.heappop(queue)
        current_pos = current['path'][-1]
        current_path = current['path']
        
        if current_pos[0] == end[0] and current_pos[1] == end[1]:
            return current_path
        
        if tuple(current_pos) in visited:
            continue
        
        visited.add(tuple(current_pos))
        
        for candidate_pos in get_candidate_positions(arr, current_pos):
            candidate_path = current_path[:] + [ candidate_pos ]
            candidate_cost = score + cost_fn(arr, current_pos, candidate_pos) + heuristic_fn(candidate_pos, end)
            
            heapq.heappush(queue, KeyDict(candidate_cost, { 'path': candidate_path }))

In [ ]:
start = np.argwhere(data == 'S')[0]
end = np.argwhere(data == 'E')[0]

baseline_path = astar(data, start, end)
baseline_node_distance = { tuple(node): (len(baseline_path) - 1) - i for i, node in enumerate(baseline_path) }

baseline_data = {
    'path': baseline_path,
    'node_distance': baseline_node_distance
}

baseline_data

In [ ]:
one_step_directions = np.array([
    [0, 1],
    [1, 0],
    [0, -1],
    [-1, 0]
])

two_step_directions = np.array([
    [0, 2],
    [2, 0],
    [0, -2],
    [-2, 0]
])

path = baseline_data['path'].copy()
speedups = {}
for idx in path:
    one_steps = one_step_directions + idx
    
    rows = one_steps[:, 0]
    cols = one_steps[:, 1]
    
    mask = data[rows, cols] == '#'
    two_steps = two_step_directions[mask] + idx

    for next_idx in [ i for i in two_steps if tuple(i) in baseline_data['node_distance']]:
        original_speed = baseline_data['node_distance'][tuple(idx)]
        new_speed = baseline_data['node_distance'][tuple(next_idx)] + 2 # +2 to account for the two steps
        
        speedup = original_speed - new_speed
        
        if speedup >= 0:
            speedups.setdefault(speedup, 0)
            speedups[speedup] += 1
    
speedups

In [ ]:
target = 100

part_a_answer = sum([ speedups[key] for key in speedups.keys() if key >= target ])
part_a_answer

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [84]:
use_sample_data = False
part='b'

In [85]:
data = sample_data if use_sample_data else test_data

In [86]:
def get_path_with_cheats(arr:np.ndarray, node_distances:dict, start:np.ndarray, end:np.ndarray, memo:dict, num_cheats:int = 20, visited:set = None):
    key = tuple(start)
    if visited is None:
        visited = set([key])

    visited.add(key)

    if key in memo:
        is_vaild = (memo[key]['available_cheats'] >= num_cheats)
        if is_vaild:
            return memo[key]['distances']

    distances = [np.inf]
    for candidate in get_candidate_positions(arr, start, remove_walls=False):
        if (tuple(candidate) in visited):
            continue

        if tuple(candidate) in node_distances:
            if arr[key] == '#':
                distances.append(node_distances[tuple(candidate)])
            continue
        
        elif candidate[0] == end[0] and candidate[1] == end[1]:
            distances += [0]
            break

        elif arr[tuple(candidate)] == '#' and num_cheats > 0:
            speedups = get_path_with_cheats(arr, node_distances, candidate, end, memo, num_cheats=num_cheats - 1, visited=visited)
            distances += speedups

    memo[key] = {
        'distances': distances,
        'available_cheats': num_cheats
    }
    
    return memo[key]['distances']

In [87]:
def get_path_speedups(arr:np.ndarray, node_distances:dict, path:list, end:np.ndarray, num_cheats:int = 20):
    speedups = {}
    memo = {}

    for node in path:
        original_speed = node_distances[tuple(node)]
        cheat_speeds = get_path_with_cheats(arr, node_distances, node, end, memo, num_cheats)

        for speed in [ s for s in cheat_speeds if s < np.inf]:
            speedup = original_speed - speed
            speedups.setdefault(speedup, 0)
            speedups[speedup] += 1
        
    return speedups

In [88]:
start = np.argwhere(data == 'S')[0]
end = np.argwhere(data == 'E')[0]

baseline_path = astar(data, start, end)
baseline_node_distance = { tuple(node): (len(baseline_path) - 1) - i for i, node in enumerate(baseline_path) }

baseline_data = {
    'path': baseline_path,
    'node_distance': baseline_node_distance
}

In [ ]:
path = np.array(baseline_data['path'].copy())
node_distances = baseline_data['node_distance'].copy()

num_cheats = 20

diffs = np.abs(path[:, np.newaxis] - path[np.newaxis, :])
distances = np.sum(diffs, axis=-1)

rows, cols = np.triu_indices(len(path), k=1)

speedups = {} 
for i, j in zip(rows, cols):
    distance = distances[i, j]
    if distance <= num_cheats:
        original_speed = node_distances[tuple(path[i])]
        new_speed = node_distances[tuple(path[j])] + distance

        speedup = original_speed - new_speed

        if speedup >= 0:
            speedups.setdefault(speedup, 0)
            speedups[speedup] += 1
        

target = 100

part_b_answer = sum([ speedups[key] for key in speedups.keys() if key >= target ])
part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)